In [1]:
total_bit = 8

qmodels_for_hls4ml_dir = "qmodels_for_hls4ml/" + f"qmodel_totalbit_{total_bit}.h5"
hls_model_output_dir  = './hls_model/lfad_qmodel_' + str(total_bit)

# Load data and define NPLL metric

In [2]:
import numpy as np

In [3]:
test_neural_data = np.load("test_neural_data.npy")
inputs2decoder_test = np.load("inputs2decoder_test.npy")

print("test_neural_data shape: ", test_neural_data.shape)
print("inputs2decoder_test shape: ", inputs2decoder_test.shape)

test_neural_data shape:  (17, 73, 70)
inputs2decoder_test shape:  (17, 73, 64)


In [4]:
import tensorflow as tf

def evaluate_NPLL(targets, pred_logrates):
    targets = tf.cast(targets, dtype=tf.float32)
    logrates = tf.cast(tf.math.log(0.01) + pred_logrates, tf.float32)  # Timestep
    npll = tf.nn.log_poisson_loss(targets=targets,log_input=logrates, compute_full_loss=True)
    results = tf.reduce_sum(npll, axis=[1, 2]) # sum up each batch seperately
    results = tf.reduce_mean(results) # batch mean

    return results.numpy() # negative possion loglikelihood

# Load qkeras model

In [5]:
from qkeras.utils import load_qmodel
from qkeras.autoqkeras.utils import print_qmodel_summary

In [6]:
model = load_qmodel(
    qmodels_for_hls4ml_dir,
    compile = False
)

In [7]:
model.summary()

Model: "lfad_for_hls4ml"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 73, 70)]     0           []                               
                                                                                                  
 EncoderRNN (QBidirectional)    (None, 128)          52224       ['encoder_input[0][0]']          
                                                                                                  
 q_act_postencoder (QActivation  (None, 128)         0           ['EncoderRNN[0][0]']             
 )                                                                                                
                                                                                                  
 DenseMean (QDense)             (None, 64)           8256        ['q_act_postencoder

In [8]:
print_qmodel_summary(model)

EncoderRNN           quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) quantized_bits(8,2,0,alpha=1) quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) quantized_bits(8,2,0,alpha=1) act=quantized_tanh(8)
q_act_postencoder    quantized_bits(8,2,alpha=1)
DenseMean            u=64 quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) 
q_act_dense_mean     quantized_bits(16,6,alpha=1)
DecoderGRU           quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) quantized_bits(8,2,0,alpha=1) recurrent act=quantized_sigmoid(8)act=quantized_tanh(8)
q_act_postdecoder    quantized_bits(16,6,alpha=1)
Dense                u=4 quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) 
q_act_postdense      quantized_bits(8,2,alpha=1)
NeuralDense          u=70 quantized_bits(8,1,0,alpha=1) quantized_bits(8,1,0,alpha=1) 



In [9]:
'''
from tensorflow.keras import Model
import tensorflow.keras.layers as tfl

input_shape = (73,70)
inputs = tfl.Input(shape=input_shape, name = 'encoder_input')
encoder_GRU = model.get_layer("EncoderRNN")
q_act_postencoder = model.get_layer("q_act_postencoder")

mid_result = encoder_GRU(inputs)
mid_result = q_act_postencoder(mid_result)
encoder_model = Model(inputs = inputs, outputs = mid_result)

encoder_model.summary()
'''

'\nfrom tensorflow.keras import Model\nimport tensorflow.keras.layers as tfl\n\ninput_shape = (73,70)\ninputs = tfl.Input(shape=input_shape, name = \'encoder_input\')\nencoder_GRU = model.get_layer("EncoderRNN")\nq_act_postencoder = model.get_layer("q_act_postencoder")\n\nmid_result = encoder_GRU(inputs)\nmid_result = q_act_postencoder(mid_result)\nencoder_model = Model(inputs = inputs, outputs = mid_result)\n\nencoder_model.summary()\n'

In [10]:
#model = encoder_model

# Convert to hls

In [11]:
import hls4ml

/home/pcslab/anaconda3/envs/lfads_hls4ml/lib/python3.8/site-packages/hls4ml/converters/__init__.py:24: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


## use 32 bit for default precision
## this will be used for accumulation bit

In [12]:
hls_default_precision = 'ap_fixed<32,16>'

In [13]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name', default_precision=hls_default_precision)

config['Model']['Strategy'] = 'Resource'
config['Model']['ReuseFactor'] = 10000000 #will use N_IN 

Interpreting Model
Topology:
{'name': 'encoder_input', 'class_name': 'InputLayer', 'data_format': 'channels_last', 'input_shape': [73, 70]}
Layer name: encoder_input, layer type: InputLayer, input shapes: [[None, 73, 70]], output shape: [None, 73, 70]
{'name': 'EncoderRNN', 'class_name': 'QBidirectional', 'inputs': ['encoder_input'], 'data_format': 'channels_last', 'activation': 'hard_tanh', 'use_bias': True, 'return_sequences': False, 'return_state': False, 'recurrent_activation': 'hard_sigmoid', 'time_major': False, 'n_timesteps': 73, 'n_in': 70, 'n_out': 128, 'initial_state': 0, 'recurrent_activation_quantizer': {'class_name': 'quantized_sigmoid', 'config': {'bits': 8, 'symmetric': False, 'use_real_sigmoid': False, 'use_stochastic_rounding': False}, 'shared_object_id': 2}, 'activation_quantizer': {'class_name': 'quantized_tanh', 'config': {'bits': 8, 'symmetric': False, 'use_stochastic_rounding': False, 'use_real_tanh': False}, 'shared_object_id': 1}, 'state_quantizer': <hls4ml.mode

In [28]:
from pprint import pprint
pprint(config)

{'LayerName': {'DecoderGRU': {'ApplyResetGate': 'after',
                              'Direction': 'forward',
                              'Precision': {'act': 'ap_fixed<32,16>',
                                            'activation': 'fixed<8,1,RND_CONV,SAT>',
                                            'bias': 'fixed<8,2>',
                                            'recr_act': 'ap_fixed<32,16>',
                                            'recurrent_activation': 'ufixed<8,0,RND_CONV,SAT>',
                                            'recurrent_bias': 'fixed<8,2>',
                                            'recurrent_weight': 'fixed<8,2>',
                                            'result': 'ap_fixed<32,16>',
                                            'shift': 'ap_fixed<32,16>',
                                            'slope': 'ap_fixed<32,16>',
                                            'state': 'fixed<8,3>',
                                            'weight': 'fixe

In [29]:
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir=hls_model_output_dir,
                                                       io_type = 'io_array_stream',
                                                       #io_type = 'io_parallel',
                                                       #backend='VivadoAccelerator', board='pynq-z2')
                                                       part='xcu55c-fsvh2892-2L-e')
hls_model.compile()

Interpreting Model
Topology:
{'name': 'encoder_input', 'class_name': 'InputLayer', 'data_format': 'channels_last', 'input_shape': [73, 70]}
Layer name: encoder_input, layer type: InputLayer, input shapes: [[None, 73, 70]], output shape: [None, 73, 70]
{'name': 'EncoderRNN', 'class_name': 'QBidirectional', 'inputs': ['encoder_input'], 'data_format': 'channels_last', 'activation': 'hard_tanh', 'use_bias': True, 'return_sequences': False, 'return_state': False, 'recurrent_activation': 'hard_sigmoid', 'time_major': False, 'n_timesteps': 73, 'n_in': 70, 'n_out': 128, 'initial_state': 0, 'recurrent_activation_quantizer': {'class_name': 'quantized_sigmoid', 'config': {'bits': 8, 'symmetric': False, 'use_real_sigmoid': False, 'use_stochastic_rounding': False}, 'shared_object_id': 2}, 'activation_quantizer': {'class_name': 'quantized_tanh', 'config': {'bits': 8, 'symmetric': False, 'use_stochastic_rounding': False, 'use_real_tanh': False}, 'shared_object_id': 1}, 'state_quantizer': <hls4ml.mode

Done


In [30]:
#pred_qkeras = model.predict(test_neural_data)
pred_qkeras = model.predict([test_neural_data, inputs2decoder_test])
npll_qkeras = evaluate_NPLL(targets=test_neural_data, pred_logrates=pred_qkeras)
print("npll_qkeras: ", npll_qkeras)

#pred_hls = hls_model.predict(test_neural_data)
pred_hls = hls_model.predict([test_neural_data, inputs2decoder_test])
npll_hls = evaluate_NPLL(targets=test_neural_data, pred_logrates=pred_hls.reshape(17,73,70))
print("npll_hls: ", npll_hls)

npll_qkeras:  1827.6609
npll_hls:  1827.6609


In [31]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(pred_qkeras.flatten(), pred_hls.flatten())

0.0

In [32]:
pred_qkeras.flatten()[:100]

array([ 0.14501953,  1.8154297 ,  2.434082  ,  3.1577148 ,  2.400879  ,
        3.663086  ,  1.8144531 ,  3.5063477 ,  3.6816406 ,  2.1904297 ,
        1.4301758 ,  0.84375   ,  1.8071289 ,  1.0380859 ,  2.8105469 ,
        2.2285156 ,  2.4394531 ,  2.7944336 ,  3.4667969 ,  2.7861328 ,
        0.88964844,  3.118164  ,  0.9243164 ,  3.1586914 ,  3.7841797 ,
        2.3457031 ,  0.8330078 ,  4.1674805 ,  3.3237305 ,  4.225586  ,
       -0.53222656,  2.262207  ,  1.9667969 ,  4.008789  ,  3.7998047 ,
        2.0385742 ,  3.8666992 ,  0.16601562,  1.315918  ,  2.6972656 ,
        3.2333984 ,  0.18017578, -1.1191406 ,  0.9604492 ,  3.234375  ,
        2.942871  ,  0.65234375,  3.2011719 ,  1.6928711 ,  2.5151367 ,
        3.519043  ,  3.519043  ,  4.25      ,  1.425293  ,  1.4199219 ,
        2.0375977 ,  1.4589844 ,  1.559082  ,  0.5361328 ,  1.5522461 ,
        1.1308594 ,  2.4760742 ,  2.828125  ,  1.4189453 ,  4.330078  ,
        3.203125  ,  3.3261719 ,  1.5439453 ,  1.8457031 ,  0.16

In [33]:
pred_hls.flatten()[:100]

array([ 0.14501953,  1.81542969,  2.43408203,  3.15771484,  2.40087891,
        3.66308594,  1.81445312,  3.50634766,  3.68164062,  2.19042969,
        1.43017578,  0.84375   ,  1.80712891,  1.03808594,  2.81054688,
        2.22851562,  2.43945312,  2.79443359,  3.46679688,  2.78613281,
        0.88964844,  3.11816406,  0.92431641,  3.15869141,  3.78417969,
        2.34570312,  0.83300781,  4.16748047,  3.32373047,  4.22558594,
       -0.53222656,  2.26220703,  1.96679688,  4.00878906,  3.79980469,
        2.03857422,  3.86669922,  0.16601562,  1.31591797,  2.69726562,
        3.23339844,  0.18017578, -1.11914062,  0.96044922,  3.234375  ,
        2.94287109,  0.65234375,  3.20117188,  1.69287109,  2.51513672,
        3.51904297,  3.51904297,  4.25      ,  1.42529297,  1.41992188,
        2.03759766,  1.45898438,  1.55908203,  0.53613281,  1.55224609,
        1.13085938,  2.47607422,  2.828125  ,  1.41894531,  4.33007812,
        3.203125  ,  3.32617188,  1.54394531,  1.84570312,  0.16

In [34]:
arr1 = pred_qkeras.flatten()
arr2 = pred_hls.flatten()
diff_indices = np.where(arr1 != arr2)

In [35]:
diff_indices

(array([], dtype=int64),)

In [36]:
# Print the indices and the differing values from both arrays
for idx in diff_indices[0]:
    print(f"Index: {idx}, arr1 value: {arr1[idx]}, arr2 value: {arr2[idx]}")

# Output testbench

In [37]:
test_neural_data_flatten = test_neural_data.reshape(test_neural_data.shape[0], -1)
print(test_neural_data_flatten.shape)

(17, 5110)


In [38]:
inputs2decoder_test_flatten = inputs2decoder_test.reshape(inputs2decoder_test.shape[0], -1)
print(inputs2decoder_test_flatten.shape)

(17, 4672)


In [39]:
# concat in dim1, as hls4ml will process the encoder input first, and then decoder
x_test = np.concatenate([test_neural_data_flatten, inputs2decoder_test_flatten], axis=1)
print(x_test.shape)

(17, 9782)


In [40]:
pred_qkeras_flatten = pred_qkeras.reshape(pred_qkeras.shape[0], -1)
print(pred_qkeras_flatten.shape)

(17, 5110)


In [41]:
# save testbench files for hls project

#write x_test to tb_input_features.dat
INPUT_FILE = hls_model_output_dir + '/tb_data/tb_input_features.dat'
np.savetxt(INPUT_FILE, x_test, delimiter=' ', fmt='%f')

#write x_test to tb_input_features.dat
OUTPUT_FILE = hls_model_output_dir + '/tb_data/tb_output_predictions.dat'
np.savetxt(OUTPUT_FILE, pred_qkeras_flatten, delimiter=' ', fmt='%f')